In [34]:
import numpy as np
import pandas as pd
import re
import collections
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [35]:
data = pd.read_csv('/home/ryu/2023text/final/upto_summarization_0603.csv')
data.head()

,title,timeline,year,month,sentences,summarization
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,before,2022,7,['거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 ...,['거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 ...
1,중부지방에 5 40 소나기 낮 최고 35도 폭염,before,2022,7,['우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개...,['폭염특보가 발표된 일부 강원 영동 중 남부와 충청권 남부지방 제주도를 중심으로 ...
2,삼성 세계 첫 3나노 양산 TSMC 잡을까,before,2022,7,"['삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다', ...","['이곳에서 3나노미터 공정이 적용된 세계 최초의 반도체가 양산되기 시작했습니다',..."
3,신한은행 ESG본부 신설 하반기 정기 인사,before,2022,7,['AI 인사 업그레이드 금융사고 예방 위해 장기근속 직원 순환근무 신한은행은 1일...,['신한은행에 따르면 이번 인사는 본점 영업점 장기근속 직원의 순환 근무를 통해 각...
4,영남이공대 미래 신산업 수요특화형 AI IoT 교육,before,2022,7,['영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에...,['영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에...


In [36]:
data.drop(['sentences'], axis=1, inplace=True)

In [39]:
data[data.duplicated(subset='summarization')].head(50)

,title,timeline,year,month,summarization
118,경남 민선 8기 개막 경제 살리기로 지방소멸 극복,before,2022,7,['민선 8기 경남지사와 교육감 시장 군수들이 오늘 취임식을 열고 4년의 임기를 시...
878,데이터 인공지능 분야 투자애로 규제개선 현장간담회 참석하는 남궁훈 카타오 대표이사,before,2022,7,[]
919,악수하는 한덕수 총리,before,2022,7,['뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부로 보내...
922,한덕수 국무총리 발언,before,2022,7,['뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부로 보내...
940,남궁훈 카카오 대표이사 파이팅,before,2022,7,[]
944,투자애로 규제개선 현장간담회의 토론 참석자들,before,2022,7,[]
969,한미 경제안보대화 첫 회의...공급망 첨단기술 협력 논의,before,2022,7,['한미 경제안보 대화 백악관에서 첫 대면 회의경제 안보 공급망 과학기술 세 분야 ...
982,발언하는 한덕수 국무총리,before,2022,7,['뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부로 보내...
988,더존 기업설명 듣는 한덕수 국무총리,before,2022,7,['뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부로 보내...
991,현장간담회 참석하는 한덕수 국무총리,before,2022,7,['뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부로 보내...


In [40]:
data = data.drop_duplicates(subset='summarization', keep='first', ignore_index=True)

In [44]:
data = data[data['summarization']!='[]']

In [45]:
len(data)

39726

In [46]:
# 댓글은 다시 리스트형으로 변환 (데이터 가져오면서 리스트를 문자열로 인식)
data['summarization']=data['summarization'].str.replace('[','').str.replace(']','').str.replace('\'','') #nouns 칼럼에 있는 []\문자 제거
data['summarization']=data['summarization'].apply(lambda x:x.split(',')) # ,를 기준으로 분리 후 리스트에 담기
data['summarization']=data['summarization'].apply(lambda x:[i.strip() for i in x]) # 리스트 내 문자 양쪽 공백제거

/tmp/ipykernel_71048/3400779417.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['summarization']=data['summarization'].str.replace('[','').str.replace(']','').str.replace('\'','') #nouns 칼럼에 있는 []\문자 제거


In [21]:
data['summarization'][39452]

['한미동맹은 대한민국의 자유와 평화를 지키고 번영을 일구어 온 중심축이었습니다',
 '잠시 뒤에 상원 의원들의 에스코트를 받아 대한민국 윤석열 대통령이 이곳에 들어와서 상하원 합동연설을 진행할 예정입니다',
 '그렇다면 러시아 북한 중국 모두 어제 한미 정상회담 이후에 발표 내용 공동성명에 대해서도 들었을 것이고 오늘 상하원 합동연설에서도 분명히 반응할 수 있을 만한 포인트가 있을 것 같은데 어떻게 대응할까요 역설적으로 이야기하자면 윤석열 대통령이 이번 국빈 방문을 출발하기 전부터 여러 가지 언론 인터뷰 기회를 통해서 같은 입장을 이미 표명했고 중국과 러시아도 거기에 대해서 반응을 보였습니다',
 '이는 한미동맹의 성공을 보여주는 것입니다',
 '한미동맹은 자유 인권 민주주의 가치로 맺어졌기 때문에 진정한 글로벌 동맹이다 이런 점을 강조했습니다',
 '그것이 아니라 자유를 지키기 위한 희생의 산물이라는 점을 강조했지 않았습니까 그리고 여러 가지 행사에서 윤석열 대통령이 취임한 이후에 했던 연설의 키워드가 자유였기 때문에 동맹과 자유를 연계시키는 그런 메시지를 전달할 것으로 예상됩니다',
 '대한민국은 이제 자유와 민주주의가 살아 숨쉬는 활력 넘치는 나라로 세계 시민의 사랑을 받고 있습니다',
 '한국 대통령으로서는 10년 만에 상하원 합동연설에 나서는 것인데요',
 '미국 상하원 합동연설',
 '연설에 앞서 상하원 의원들의 박수 소리도 좀 더 커지고 있습니다',
 '예상했던 시간 30분을 훌쩍 넘기고 40분을 넘겨서 45분 정도 미 의회 상하원 합동연설 윤석열 대통령의 연설이 있었고요',
 '아직도 지금 미 의회 본회의장에서는 윤석열 대통령과 상하원 의원들의 자연스러운 인사 또 접촉이 있고요',
 '어떤 상황이라고 볼 수 있을까요 지금 의원들이 아마 오늘 합동연설에 참가한 상하원 의원들을 호명들을 하고 있는 것 같아요',
 '그런 얘기를 했는데 70년 동안의 동맹이 한국의 번영 그리고 민주주의의 발전에 어떻게 기여했는지를 치하하는 그런 대목도 윤석열 대통령의

In [ ]:
data[data['summarization'].apply(len)>=100]

In [49]:
len(data[data['summarization'].apply(len)>=100])

51

In [50]:
data = data[data['summarization'].apply(len)<100]

In [51]:
data[data['summarization'].apply(len)==99]

,title,timeline,year,month,summarization
9324,미네르바 설립자 첨단학과 증설 기술 바뀌면 무의미,before,2022,9,[벤 넬슨 미네르바대 설립자 인터뷰2024년 한국에 미네르바대 설립 추진AI 기술 ...
21753,당신이 잠든 사이에도 도로 관리는 쉬지 않습니다,after,2022,12,"[서울시 서부도로관리사업소 제공 눈도 많이 와서 제설작업도 많이 하셨을 것 같은데,..."
27739,야놀자는 왜 대통령 중동 순방에 동행했을까,after,2023,2,[야놀자는 어떻게 숙박 레저 중개 플랫폼을 구축할 수 있었나 스프 야놀자가 처음에 ...


In [52]:
# 각 행의 list를 칼럼으로 쪼개기
split_df = data['summarization'].apply(lambda x: pd.Series(x))
split_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,이씨의 논문과 센스타임의 논문이 동일하게 코넬대 연구진의 논문을 인용했다고 하더라도...,이 또 다른 논문에 표절 의혹이 제기된 건 지난달 27일 서울대 온라인 커뮤니티에서다,빨간색으로 표시된 부분은 홍콩계 AI기업 센스타임 연구자 등이 발표한 논문의 문장을...,빨간색 표시된 문장이 서울대 연구팀 논문과 일치하는 부분,앞서 지난달 24일 표절 의혹이 제기된 논문과 마찬가지로 윤성로 서울대 전기정보공학...,44단어 두 문장 다른 논문에서 짜깁기1일 한국경제신문 취재를 종합하면 추가적으로 ...,서울대 표절 조사 진행 중윤 교수 연구팀에게는 지난달 24일 첫번째 표절 의혹이 제...,빨간색으로 표시된 부분은 중국 장쑤대 연구자 등이 발표한 논문의 문장을 대부분 베껴왔다,연구윤리를 따르면다면 이씨는 센스타임의 문장을 그대로 가져오는 것이 아니라 코넬대 ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,폭염특보가 발표된 일부 강원 영동 중 남부와 충청권 남부지방 제주도를 중심으로 최고...,경기 남동 내륙과 강원 남부 내륙 산지 충남권 남동 내륙 충북 북부 경상권 내륙에 ...,안쪽 먼바다의 파고는 동해 xx5 서해 남해 xx5 xx0m로 예상된다,경기 북부와 강원 북부 내륙 제주도 산지에는 오전까지 5 미만의 비가 내리겠다,우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개를 ...,인공지능이 쓴 초고와 기상청 데이터 등을 토대로 가 최종 기사를 완성했으며 데스킹을...,이날 오전 5시 현재 기온은 서울 2xx9도 인천 2xx7도 수원 2xx7도 춘천 ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,이곳에서 3나노미터 공정이 적용된 세계 최초의 반도체가 양산되기 시작했습니다,삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다,TSMC도 하반기에 3나노 공정 양산을 예고하고 있어서 경쟁에 불이 붙을 것으로 보입니다,삼성전자의 첨단 반도체가 생산되는 화성공장,3나노 공정이 적용된 반도체는 스마트폰 자율주행 인공지능 같이 고성능이 필요한 곳에...,회로와 회로 사이의 폭을 이만큼 좁혔습니다,MBC 뉴스는 24시간 여러분의 제보를 기다립니다,파운드리란 CPU나 스마트폰의 AP 같은 주로 핵심 반도체를 위탁생산하는 분야입니다,경쟁사인 대만 TSMC보다 반년 정도 앞섰다고 하는데 시장에서 판도를 뒤집을 수 있...,경쟁력이 있으려면 불량품 없이 잘 생산하는 정도를 나타내는 수율 이 높아야 합니다,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
split_df.stack()

0      0    거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
       1    이씨의 논문과 센스타임의 논문이 동일하게 코넬대 연구진의 논문을 인용했다고 하더라도...
       2      이 또 다른 논문에 표절 의혹이 제기된 건 지난달 27일 서울대 온라인 커뮤니티에서다
       3    빨간색으로 표시된 부분은 홍콩계 AI기업 센스타임 연구자 등이 발표한 논문의 문장을...
       4                      빨간색 표시된 문장이 서울대 연구팀 논문과 일치하는 부분
                                  ...                        
39726  1    어린이 날 연휴 로봇마차 퍼레이드 솜 사탕 로봇 라면 끓이는 로봇 도 선보여로봇 소...
       2    로봇랜드는 다음달 부터 중앙 드림광장에서 풍선과 비눗방울을 활용한 마술 벌룬 버블쇼...
       3    로봇특화 교육 및 진로체험으로 구성된 로봇체험교실 에서는 로봇랜드 대표 놀이시설인 ...
       4    특히 어린이날 연휴 동안 마칭밴드가 로봇마차와 함께 행진하는 퍼레이드를 벌이고 로봇...
       5    류명현 경남도 산업통상국장은 매년 리얼로봇을 추가 도입하고 다양한 로봇 콘텐츠를 보...
Length: 397338, dtype: object

In [10]:
# 인덱스 제거
split_df.stack().reset_index(level=1, drop=True)

0        거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
0        이씨의 논문과 센스타임의 논문이 동일하게 코넬대 연구진의 논문을 인용했다고 하더라도...
0          이 또 다른 논문에 표절 의혹이 제기된 건 지난달 27일 서울대 온라인 커뮤니티에서다
0        빨간색으로 표시된 부분은 홍콩계 AI기업 센스타임 연구자 등이 발표한 논문의 문장을...
0                          빨간색 표시된 문장이 서울대 연구팀 논문과 일치하는 부분
                               ...                        
40002    어린이 날 연휴 로봇마차 퍼레이드 솜 사탕 로봇 라면 끓이는 로봇 도 선보여로봇 소...
40002    로봇랜드는 다음달 부터 중앙 드림광장에서 풍선과 비눗방울을 활용한 마술 벌룬 버블쇼...
40002    로봇특화 교육 및 진로체험으로 구성된 로봇체험교실 에서는 로봇랜드 대표 놀이시설인 ...
40002    특히 어린이날 연휴 동안 마칭밴드가 로봇마차와 함께 행진하는 퍼레이드를 벌이고 로봇...
40002    류명현 경남도 산업통상국장은 매년 리얼로봇을 추가 도입하고 다양한 로봇 콘텐츠를 보...
Length: 405576, dtype: object

In [54]:
# 데이터프레임 형태로 변환
split_df = split_df.stack().reset_index(level=1, drop=True).to_frame('split_sent')
split_df

,split_sent
0,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
0,이씨의 논문과 센스타임의 논문이 동일하게 코넬대 연구진의 논문을 인용했다고 하더라도...
0,이 또 다른 논문에 표절 의혹이 제기된 건 지난달 27일 서울대 온라인 커뮤니티에서다
0,빨간색으로 표시된 부분은 홍콩계 AI기업 센스타임 연구자 등이 발표한 논문의 문장을...
0,빨간색 표시된 문장이 서울대 연구팀 논문과 일치하는 부분
...,...
39726,어린이 날 연휴 로봇마차 퍼레이드 솜 사탕 로봇 라면 끓이는 로봇 도 선보여로봇 소...
39726,로봇랜드는 다음달 부터 중앙 드림광장에서 풍선과 비눗방울을 활용한 마술 벌룬 버블쇼...
39726,로봇특화 교육 및 진로체험으로 구성된 로봇체험교실 에서는 로봇랜드 대표 놀이시설인 ...
39726,특히 어린이날 연휴 동안 마칭밴드가 로봇마차와 함께 행진하는 퍼레이드를 벌이고 로봇...


In [55]:
# 기존의 데이터와 병합
data = data.merge(split_df, left_index=True, right_index=True, how='left')

In [ ]:
data.head(30)

In [59]:
# 인덱스를 기사 번호 칼럼으로 변경
data = data.reset_index().rename(columns={"index": "news_idx"})

In [60]:
len(data)

397338

In [61]:
data['split_sent'].isnull().sum()

0

In [62]:
data.head()

,news_idx,title,timeline,year,month,summarization,split_sent
0,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,before,2022,7,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
1,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,before,2022,7,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...,이씨의 논문과 센스타임의 논문이 동일하게 코넬대 연구진의 논문을 인용했다고 하더라도...
2,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,before,2022,7,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...,이 또 다른 논문에 표절 의혹이 제기된 건 지난달 27일 서울대 온라인 커뮤니티에서다
3,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,before,2022,7,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...,빨간색으로 표시된 부분은 홍콩계 AI기업 센스타임 연구자 등이 발표한 논문의 문장을...
4,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,before,2022,7,[거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인...,빨간색 표시된 문장이 서울대 연구팀 논문과 일치하는 부분


In [63]:
data.to_csv('/home/ryu/2023text/final/split_sum_sent_original.csv', index=False, encoding='utf-8')

In [ ]:
# import pandas as pd
# d = pd.read_csv('/home/ryu/2023text/final/split_sum_sent_original.csv')
# d.drop(['summarization'], axis=1, inplace=True)
# d.to_csv('/home/ryu/2023text/final/split_sent_mj_request.csv', index=False)

In [64]:
data.drop(['title', 'summarization'], axis=1, inplace=True)

In [65]:
data.head()

,news_idx,timeline,year,month,split_sent
0,0,before,2022,7,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
1,0,before,2022,7,이씨의 논문과 센스타임의 논문이 동일하게 코넬대 연구진의 논문을 인용했다고 하더라도...
2,0,before,2022,7,이 또 다른 논문에 표절 의혹이 제기된 건 지난달 27일 서울대 온라인 커뮤니티에서다
3,0,before,2022,7,빨간색으로 표시된 부분은 홍콩계 AI기업 센스타임 연구자 등이 발표한 논문의 문장을...
4,0,before,2022,7,빨간색 표시된 문장이 서울대 연구팀 논문과 일치하는 부분


In [67]:
data['split_sent'][0]

'거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공지능 연구팀의 표절 사실이 드러난 가운데 같은 연구팀 논문에 또다른 표절 의혹이 제기됐다'

In [98]:
data[data.duplicated(subset='split_sent')].head(50)

,news_idx,timeline,year,month,split_sent
118,14,before,2022,7,콴다는 이번 개편을 통해 풀이 검색 앱 에서 오답노트 커뮤니티 기능 등 학습 전반을...
440,40,before,2022,7,인공지능이 쓴 초고와 기상청 데이터 등을 토대로 가 최종 기사를 완성했으며 데스킹을...
767,71,before,2022,7,한경라씨로 앱을 내려 받으면 매일 시간대별로 특징적인 종목 정보를 받아 볼 수 있습니다
768,71,before,2022,7,한경라씨로는 인공지능 알고리즘을 기반으로 상장 종목들의 시세 수급 시황 등을 분석합니다
879,85,before,2022,7,이러한 디지털 대전환 시대를 맞아 부산시 부경대 네이버가 함께 소상공인 디지털 전환...
908,89,before,2022,7,부경대는 데이터를 토대로 대학생과 소상공인 매칭을 통해 맞춤형 브랜드 전략 수립과 ...
936,94,before,2022,7,인공지능이 쓴 초고와 기상청 데이터 등을 토대로 가 최종 기사를 완성했으며 데스킹을...
938,94,before,2022,7,바다의 물결은 동해 서해 남해 앞바다에서 xx5 xx0m로 일겠다
940,94,before,2022,7,안쪽 먼바다의 파고는 동해 서해 남해 xx5 xx5m로 예상된다
945,95,before,2022,7,이번 사업 선정으로 인하대는 3년간 약 5억4000만원을 지원받아 산업 간 융 복합...


In [77]:
count = 0 
# '기대';7469, '긍정';974, '성과';8200
# '위협';1871, '감소';2398, '걱정';435, '대체';1977, '공포';228, '불안';586
for s in range(len(data)):
    if '불안' in data['split_sent'][s]:
        count += 1
        print(data['split_sent'][s])

print(count)

관련해 군사 안보 전문가인 김종대 전 정의당 의원은 30일 MBC라디오 방송과의 대담에서 윤 대통령의 나토 연설에 대해 나토의 중국 견제 의지에 적극적인 동참 의사를 피력한 것 이라며 이렇게까지 꼭 했어야 되는지 이런 대통령의 말은 사실 좀 불안하다 고 지적했다 그는 회담 시작 전부터 이미 경고하고 나선 입장이기 때문에 특히 호주 뉴질랜드 한국 일본 네 나라에 대해서는 이미 대책 수립에 들어갔다 며 제일 늦게 반중전선에 뛰어든 한국에 대한 서운함이 제일 클 것이고 앞으로 한중관계에서 외교 경제 분야에서 상당히 어떤 위기가 발생할 가능성에 대비해야 될 것 같다 고 했다
윤 대통령은 북한의 핵 미사일 위협이 고도화되고 국제정세의 불안정이 커진 상황에서 한미일 협력의 중요성은 더욱 커졌다 며 3국 협력 강화 의지 를 강조했고 바이든 대통령은 한미일 3각 협력은 우리의 공통목표 달성에 매우 중요하다 며 그 중에는 한반도의 완전한 비핵화 자유롭고 개방된 인도 태평양 지역이 포함돼 있다 고 강조했다
앞서 북한은 29일 오전 관영 조선중앙통신을 통해 윤 대통령의 나토 참석을 공개적으로 비난하며 남조선 당국은 화난의 근원인 나토의 검은 손을 잡음으로써 매우 고통스러운 중증 안보위기를 경과하게 될 것이며 치유 불능의 안보불안 후유증에 시달리게 될 것 이라고 경고했다
코로나19 시대에 사람들은 그 어느 때보다도 공간과 공기를 타인과 공유하는 것에 대해 날카롭게 의식하고 불안을 느끼게 되었다
냄새라는 것은 신체에 대한 불안과 밀접한 관련이 있다
이런 생각을 하면 공기에 대한 불안이 줄어들게 된다
인간은 죽음에 대한 불안감을 가지고 있어서 모든 것을 시간의 흐름으로부터 정지시키고 싶어한다
일반 투자자 보호조치 시행으로 수급 불안 우려가 커졌기 때문이다
따라서 생산 요소 확보에 대한 불안감이 생겼고 이로 인한 가수요가 인플레이션을 증폭시켰다
경제지 포춘은 테슬라의 프리몬트 공장에서 일하는 직원들이 부족한 주차 공간 불안정한 인터넷 연결 등으로 불편을 겪고 있다 고 전했다
실제 기업 

In [ ]:
count = 0 
# '기대';7469, '긍정';974, '성과';8200
# '위협';1871, '감소';2398, '걱정';435, '대체';1977, '공포';228, '불안';586
for s in range(len(data)):
    if '불안' in data['split_sent'][s]:
        count += 1
        print(data['split_sent'][s])

print(count)

In [82]:
pos1 = data[data['split_sent'].str.contains('기대')]
pos2 = data[data['split_sent'].str.contains('긍정')]
pos3 = data[data['split_sent'].str.contains('성과')]
neg1 = data[data['split_sent'].str.contains('위협')]
neg2 = data[data['split_sent'].str.contains('감소')]
neg3 = data[data['split_sent'].str.contains('걱정')]

In [83]:
pos_df = pd.concat([pos1, pos2, pos3], axis=0, ignore_index=True)
neg_df = pd.concat([neg1, neg2, neg3], axis=0, ignore_index=True)

In [102]:
print(len(pos_df))
print(len(neg_df))

16643
4704


In [85]:
pos_sample = pos_df.sample(1100)
neg_sample = neg_df.sample(1100)
random_sample = data.sample(1100)

In [86]:
labeling_sample = pd.concat([pos_sample, neg_sample, random_sample], axis=0, ignore_index=True)
labeling_sample.head()

,news_idx,timeline,year,month,split_sent
0,32728,after,2023,3,이날 신뢰 가능한 AI 인공지능 비즈니스와 사람 을 주제로 기조강연을 한 차미영 K...
1,26592,after,2023,1,SK하이닉스는 이 같은 프로그램으로 미래 반도체 인재의 저변을 확대할 수 있을 것으...
2,3332,before,2022,7,구현모 KT 대표가 28일 2022년 상반기 KT그룹 혁신성과 공유회 에서 인사말을...
3,37470,after,2023,4,올해 취임 2년 차 본격적인 성과를 내야 하는 그가 은행권 최초로 데이터 기반 인재...
4,10781,before,2022,9,대웅제약은 이번 계약으로 연구 기간을 대폭 단축하는 한편 항암신약 개발의 성공 가능...


In [87]:
len(labeling_sample)

3300

In [93]:
labeling_sample[labeling_sample.duplicated(subset='split_sent')]

,news_idx,timeline,year,month,split_sent
179,12760,before,2022,10,경영 성과도 시장의 기대를 넘어서고 있다
437,3870,before,2022,8,이복기 원티드랩 대표는 원티드는 차별화된 데이터와 기술력을 기반으로 채용 시장을 선...
467,21658,after,2022,12,장 차관은 대구가 대동모빌리티 지역 협력기업 간 공동연구개발을 통해 스마트 모빌리티...
581,39301,after,2023,4,이준영 학생은 관심 기술이던 AI를 측정 분야에 적용할 수 있는 결과로 뜻깊고 많은...
806,20079,after,2022,12,챗GPT와 대화해보니코드 짜주고 편지 써주고 부정확한 답변하기도 검색의 시대 끝났다...
846,28534,after,2023,2,가격도 30 50 가량 높아 메모리반도체 업계의 수익성 확대도 기대할 수 있다
852,18282,before,2022,11,사람과 산업을 이롭게 라는 주제 하에 개최되는 이번 데모데이는 하반기 육성 프로그램...
906,18117,before,2022,11,스타트업의 육성 성과를 알리고 사업 협력 및 투자 유치 기회를 제공하는 자리다
930,15420,before,2022,11,양 장관은 북한의 대량살상무기확보 및 사용을 방지하고 WMD 위협을 감소시켜 동맹의...
1003,39292,after,2023,4,이준영 학생은 관심 기술이던 AI를 측정 분야에 적용할 수 있는 결과로 뜻깊고 많은...


In [95]:
labeling_sample = labeling_sample.drop_duplicates(subset='split_sent', keep='first', ignore_index=True)
labeling_sample.head()

,news_idx,timeline,year,month,split_sent
0,32728,after,2023,3,이날 신뢰 가능한 AI 인공지능 비즈니스와 사람 을 주제로 기조강연을 한 차미영 K...
1,26592,after,2023,1,SK하이닉스는 이 같은 프로그램으로 미래 반도체 인재의 저변을 확대할 수 있을 것으...
2,3332,before,2022,7,구현모 KT 대표가 28일 2022년 상반기 KT그룹 혁신성과 공유회 에서 인사말을...
3,37470,after,2023,4,올해 취임 2년 차 본격적인 성과를 내야 하는 그가 은행권 최초로 데이터 기반 인재...
4,10781,before,2022,9,대웅제약은 이번 계약으로 연구 기간을 대폭 단축하는 한편 항암신약 개발의 성공 가능...


In [100]:
len(labeling_sample)

3000

In [99]:
labeling_sample = labeling_sample.sample(3000)

In [101]:
labeling_sample.to_csv('/home/ryu/2023text/final/need_labels.csv', index=False, encoding='utf-8')